In [1]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
from pathlib import Path
from tqdm import tqdm
from pathlib import Path

dir_with_files = Path.cwd() / 'depended_countries' 
dir_with_graph =  Path.cwd() / 'graph' 

In [2]:
ua = UserAgent()
bad_requests = []

def execute_query(qquery, entity_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(qid = f'wd:{entity_id}'))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def instanceOf(entity_id, f = 0):
        res = []
        if f > 5:
            global bad_requests
            bad_requests.append(entity_id)
            print('Bad request ', entity_id)
            return None
        try:
            results= execute_query(bookquery.instance_of_with_entities, entity_id)
            for result in results:
                entities = {}
                entities['class'] = result['qid']['value'].split('/')[-1]
                entities['classlabel'] = result['qidLabel']['value']
                entities['subclass'] = result['instanceOf']['value'].split('/')[-1]
                entities['label'] = result['instanceOfLabel']['value']
                res.append(entities)
            return pd.DataFrame(data = res, columns = ['class', 'classlabel' ,'subclass', 'label'])
        except Exception as e:
            print(e)
            time.sleep(5)
            print('Try again ', entity_id, f)
            return entities_sparqul(entity_id, f + 1)
        
def subclasses(entity_id, f = 0):
        res = []
        if f > 5:
            global bad_requests
            bad_requests.append(entity_id)
            print('Bad request ', entity_id)
            return None
        try:
            results= execute_query(bookquery.subclass_of_with_entities, entity_id)
            for result in results:
                entities = {}
                entities['class'] = result['qid']['value'].split('/')[-1]
                entities['classlabel'] = result['qidLabel']['value']
                entities['subclass'] = result['subclass']['value'].split('/')[-1]
                entities['label'] = result['subclassLabel']['value']
                res.append(entities)
            return pd.DataFrame(data = res, columns = ['class', 'classlabel' ,'subclass', 'label'])
        except Exception as e:
            print(e)
            time.sleep(5)
            print('Try again ', entity_id, f)
            return entities_sparqul(entity_id, f + 1)

In [3]:
df = pd.read_excel(dir_with_files / 'instances_of_countries_list.xlsx')
new_df = df[(df.Check == 1) | (df.Check == 0)]
list(new_df['instaceOf'])

['Q133156',
 'Q333542',
 'Q1462047',
 'Q465613',
 'Q1371288',
 'Q161243',
 'Q164142',
 'Q426759',
 'Q1138279',
 'Q15649510',
 'Q1351282',
 'Q1151405',
 'Q3241965',
 'Q182547',
 'Q2577883',
 'Q2327385',
 'Q223832',
 'Q21479969',
 'Q18920569',
 'Q17259945',
 'Q41792036',
 'Q985073',
 'Q44565',
 'Q325261',
 'Q208164',
 'Q188686',
 'Q56167919',
 'Q26830017',
 'Q15649504',
 'Q543813',
 'Q26879763',
 'Q7448101']

In [4]:
for itter in tqdm(range(5)):
    res = []
    exl = pd.read_excel(dir_with_graph /'depended_countries_instance_without_0.xlsx', sheet_name= f'+{itter}_зависимые_территории') #читаешь эксель где все qid по которым надо идти наверх
    for i in exl['subclass']:
        res.append(instanceOf(i))
    final_result = pd.concat(res)
    with pd.ExcelWriter(dir_with_graph /'depended_countries_instance_without_0.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
            final_result.to_excel(writer, sheet_name = f'+{itter+1}_зависимые_территории' , encoding='utf-8-sig' , index = False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.63s/it]


In [5]:
for itter in tqdm(range(5)):
    res = []
    exl = pd.read_excel(dir_with_graph /'depended_countries_subclasses_without_0.xlsx', sheet_name= f'+{itter}_зависимые_территории') #читаешь эксель где все qid по которым надо идти наверх
    for i in exl['subclass']:
        res.append(subclasses(i))
    final_result = pd.concat(res)
    with pd.ExcelWriter(dir_with_graph /'depended_countries_subclasses_without_0.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
            final_result.to_excel(writer, sheet_name = f'+{itter+1}_зависимые_территории' , encoding='utf-8-sig' , index = False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:16<00:00, 27.23s/it]


In [7]:
for itter in tqdm(range(5)):
    res = []
    exl = pd.read_excel(dir_with_graph /'depended_countries_instance.xlsx', sheet_name= f'+{itter}_зависимые_территории') #читаешь эксель где все qid по которым надо идти наверх
    for i in exl['subclass']:
        res.append(instanceOf(i))
    final_result = pd.concat(res)
    with pd.ExcelWriter(dir_with_graph /'depended_countries_instance.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
            final_result.to_excel(writer, sheet_name = f'+{itter+1}_зависимые_территории' , encoding='utf-8-sig' , index = False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:25<00:00,  5.13s/it]


In [8]:
for itter in tqdm(range(5)):
    res = []
    exl = pd.read_excel(dir_with_graph /'depended_countries_subclasses.xlsx', sheet_name= f'+{itter}_зависимые_территории') #читаешь эксель где все qid по которым надо идти наверх
    for i in exl['subclass']:
        res.append(subclasses(i))
    final_result = pd.concat(res)
    with pd.ExcelWriter(dir_with_graph /'depended_countries_subclasses.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
            final_result.to_excel(writer, sheet_name = f'+{itter+1}_зависимые_территории' , encoding='utf-8-sig' , index = False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:05<00:00, 37.07s/it]


In [ ]:
# for itter in range(5):
#     res = []
#     exl = pd.read_excel('graph\graph.xlsx', sheet_name= f'+{itter}_море') #читаешь эксель где все qid по которым надо идти наверх
#     for i in exl['subclass']:
#         res.append(entities_sparqul(i))
#     final_result = pd.concat(res)
#     with pd.ExcelWriter(r'graph\graph.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
#             final_result.to_excel(writer, sheet_name = f'+{itter+1}_море' , encoding='utf-8-sig' , index = False)